In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

from math import sqrt
from scipy import stats
from statsmodels.formula.api import ols
from sklearn.metrics import mean_squared_error, explained_variance_score
from sklearn.linear_model import LinearRegression, TweedieRegressor
from sklearn.feature_selection import RFE, SelectKBest, f_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, QuantileTransformer

import env
import acquire_zillow
import prepare
import wrangle_clustering



# Data Acquisition 

- #### Acuired the Zillow data from the Sql database 
- #### Saved the data in a csv file
- #### Stored the data in a in a function as a dataframe 
- #### The original data has 52440 rows and 6 columns
- #### The cleaned data has 49000 rows and 6 columns 

# Preparation

### Steps taken for a clean dataset
1. #### Adressed missing values by dropping them since they were less than 10% of the initial data 

2. #### In order to simulate an ordinary homebuyer I took care of the outliers by:
     - removing houses over 60000 sqfeet, below 70feet
     - removing houses costing over 1.2 million in tax dollar amount 
     - removing rows with houses over 6 bedrooms and 6 bathrooms 
     
3. #### Converted fips column to categorical and changed the names to county names for readability purposes 
     - Ventura for  "6111"
     - Orange for"6059
     - Los_Angeles for "6037"

4. #### Changed the column names to make more sense in English:
     - bathrooms for'bathroomcnt'
     - bedrooms for 'bedroomcnt'
     - squarefeet for 'calculatedfinishedsquarefeet'
     - total_taxes for 'taxvaluedollarcnt'
     - county for fibs
     
5. #### Dropped the tax_amount column to avoid data leakage during modeling

6. #### Created a function that splits the data into train, validate and test dataframes

7. #### Created a function that that scaled the data for modeling purposes


In [2]:
# acquiring and preparing data. This function gets data from sql into a CSV file that is clean and prepared for exploration
df = wrangle_clustering.wrangle_zillow()
df.head()


,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,county,longitude,latitude,regionidzip,yearbuilt,taxvaluedollarcnt,max_transactiondate,logerror,propertylandusedesc,month,county_Los_Angeles,county_Orange,county_Ventura
0,14297519,3.5,4.0,3100.0,Orange,-117869207.0,33634931.0,96978.0,1998.0,1023282.0,2017-01-01,0.025595,Single Family Residential,1,0,1,0
1,17052889,1.0,2.0,1465.0,Ventura,-119281531.0,34449266.0,97099.0,1967.0,464000.0,2017-01-01,0.055619,Single Family Residential,1,0,0,1
2,14186244,2.0,3.0,1243.0,Orange,-117823170.0,33886168.0,97078.0,1962.0,564778.0,2017-01-01,0.005383,Single Family Residential,1,0,1,0
3,12177905,3.0,4.0,2376.0,Los_Angeles,-118240722.0,34245180.0,96330.0,1970.0,145143.0,2017-01-01,-0.103410,Single Family Residential,1,1,0,0
4,12095076,3.0,4.0,2962.0,Los_Angeles,-118179824.0,34145202.0,96293.0,1950.0,773303.0,2017-01-01,-0.001011,Single Family Residential,1,1,0,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48983 entries, 0 to 52441
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      48983 non-null  int64  
 1   bathroomcnt                   48983 non-null  float64
 2   bedroomcnt                    48983 non-null  float64
 3   calculatedfinishedsquarefeet  48983 non-null  float64
 4   county                        48983 non-null  object 
 5   longitude                     48983 non-null  float64
 6   latitude                      48983 non-null  float64
 7   regionidzip                   48983 non-null  float64
 8   yearbuilt                     48983 non-null  float64
 9   taxvaluedollarcnt             48983 non-null  float64
 10  max_transactiondate           48983 non-null  object 
 11  logerror                      48983 non-null  float64
 12  propertylandusedesc           48983 non-null  object 
 13  m

In [4]:
df = df.replace({"regionidzip":{399675: 99675}})
df['regionidzip'] = df['regionidzip'].astype("category")


## Features to combine

In [5]:
df["price_sqft"] = df.taxvaluedollarcnt/df.calculatedfinishedsquarefeet


In [6]:
df["age"] =2017-df.yearbuilt

In [7]:
df["price_bath"] = df.taxvaluedollarcnt/ df.bathroomcnt

In [8]:
df.join(df.groupby("regionidzip").taxvaluedollarcnt.mean(), on="regionidzip", rsuffix = "_zone")
# df.join(df.groupby('month')['A'].sum(), on='month', rsuffix='_r')


,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,county,longitude,latitude,regionidzip,yearbuilt,taxvaluedollarcnt,...,logerror,propertylandusedesc,month,county_Los_Angeles,county_Orange,county_Ventura,price_sqft,age,price_bath,taxvaluedollarcnt_zone
0,14297519,3.5,4.0,3100.0,Orange,-117869207.0,33634931.0,96978.0,1998.0,1023282.0,...,0.025595,Single Family Residential,1,0,1,0,330.090968,19.0,292366.285714,887018.295775
1,17052889,1.0,2.0,1465.0,Ventura,-119281531.0,34449266.0,97099.0,1967.0,464000.0,...,0.055619,Single Family Residential,1,0,0,1,316.723549,50.0,464000.000000,523075.173653
2,14186244,2.0,3.0,1243.0,Orange,-117823170.0,33886168.0,97078.0,1962.0,564778.0,...,0.005383,Single Family Residential,1,0,1,0,454.366854,55.0,282389.000000,636787.602667
3,12177905,3.0,4.0,2376.0,Los_Angeles,-118240722.0,34245180.0,96330.0,1970.0,145143.0,...,-0.103410,Single Family Residential,1,1,0,0,61.087121,47.0,48381.000000,465065.754902
4,12095076,3.0,4.0,2962.0,Los_Angeles,-118179824.0,34145202.0,96293.0,1950.0,773303.0,...,-0.001011,Single Family Residential,1,1,0,0,261.074612,67.0,257767.666667,722194.729412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52437,11000655,2.0,2.0,1286.0,Los_Angeles,-118282383.0,34245368.0,96284.0,1940.0,354621.0,...,0.020615,Single Family Residential,9,1,0,0,275.755054,77.0,177310.500000,354013.143791
52438,17239384,2.0,4.0,1612.0,Ventura,-118706327.0,34300140.0,97116.0,1964.0,67205.0,...,0.013209,Single Family Residential,9,0,0,1,41.690447,53.0,33602.500000,398849.164384
52439,12773139,1.0,3.0,1032.0,Los_Angeles,-118038169.0,34040895.0,96480.0,1954.0,49546.0,...,0.037129,Single Family Residential,9,1,0,0,48.009690,63.0,49546.000000,219202.984848
52440,12826780,2.0,3.0,1762.0,Los_Angeles,-117996709.0,33937685.0,96171.0,1955.0,522000.0,...,0.007204,Single Family Residential,9,1,0,0,296.254257,62.0,261000.000000,307329.656934


In [9]:
df_dropped_cols = df.drop(columns = ["county","propertylandusedesc"])
df_dropped_cols.head()

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,longitude,latitude,regionidzip,yearbuilt,taxvaluedollarcnt,max_transactiondate,logerror,month,county_Los_Angeles,county_Orange,county_Ventura,price_sqft,age,price_bath
0,14297519,3.5,4.0,3100.0,-117869207.0,33634931.0,96978.0,1998.0,1023282.0,2017-01-01,0.025595,1,0,1,0,330.090968,19.0,292366.285714
1,17052889,1.0,2.0,1465.0,-119281531.0,34449266.0,97099.0,1967.0,464000.0,2017-01-01,0.055619,1,0,0,1,316.723549,50.0,464000.000000
2,14186244,2.0,3.0,1243.0,-117823170.0,33886168.0,97078.0,1962.0,564778.0,2017-01-01,0.005383,1,0,1,0,454.366854,55.0,282389.000000
3,12177905,3.0,4.0,2376.0,-118240722.0,34245180.0,96330.0,1970.0,145143.0,2017-01-01,-0.103410,1,1,0,0,61.087121,47.0,48381.000000
4,12095076,3.0,4.0,2962.0,-118179824.0,34145202.0,96293.0,1950.0,773303.0,2017-01-01,-0.001011,1,1,0,0,261.074612,67.0,257767.666667


In [10]:
# plt.figure(figsize=(14,8))
# sns.scatterplot(x='longitude', y='latitude', hue='regionidzip', data=df[(df.regionidzip == 97319.0) |(df.regionidzip == 97318.0) |(df.regionidzip == 97329.0) |(df.regionidzip == 97328.0) | (df.regionidzip== 97118)], palette = ['blue','red','green','orange','black'])

In [11]:
#this code is a function in the prepare file that takes in a df and returns split data(train validate and test)
train, validate, test = wrangle_clustering.split_data(df)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [ ]:
#this code shows a sample of the dataframe
train.head()

# Exploration


In [ ]:
hfjygkglkh

-  According to the features in the zillow dataset, there were atleast 58 features with plenty of information. However, in the end I chose to stick to 6 features. 
-  These features were bedrooms, bathrooms, squarefeet, total taxes yearbuilt and county.
-  The following questions arose after doing alot of domain research:

1. #### does the number of bedrooms and bathrooms in a house increase the cost of the home?
2. #### is the price of the house influenced by the county/ location?
3. #### does the age of the house matter when it comes to the price? 
4. #### does the size of the house in sqfeet influence the price?


### 1. Does the number of bedrooms and bathrooms in a house increase the cost of the home


In [ ]:
target = train.total_taxes
train_drop = train.drop(columns=["total_taxes","squarefeet","yearbuilt","county"])

# I use a for loop to boxplot my independent variables against our dependent one:
for col in train_drop:
    sns.boxplot(x=train_drop[col], y=target, data=train_drop)
    plt.ylabel('total_taxes')
    plt.xlabel(col)
    plt.legend()
    plt.tight_layout()
    plt.show()
# I use a for loop to barplots my independent variables against our dependent one:
for col in train_drop:
    sns.barplot(x=train_drop[col], y=target, data=train_drop)
    plt.ylabel('total_taxes')
    plt.xlabel(col)
    plt.legend()
    plt.tight_layout()
    plt.show()  
    

## Takeaways
### 1. Does the number of bedrooms and bathrooms in a house increase the cost of the home
#### it appears that the numberof bedrooms have a positive relationship with the cost of the price of house
#### additionally according to the charts the higher the number of bathrooms the higher the price

### 2. Is the price of the house influenced by the county/ location?


In [ ]:
target = train.total_taxes
train_drop = train.drop(columns=["total_taxes","squarefeet","yearbuilt","bedrooms","bathrooms"])

# I use a for loop to boxplot my independent variables against our dependent one:
for col in train_drop:
    sns.boxplot(x=train_drop[col], y=target, data=train_drop)
    plt.ylabel('total_taxes')
    plt.xlabel(col)
    plt.legend()
    plt.tight_layout()
    plt.show()
    

# I use a for loop to boxplot my independent variables against our dependent one:
for col in train_drop:
    sns.barplot(x=train_drop[col], y=target, data=train_drop)
    plt.ylabel('total_taxes')
    plt.xlabel(col)
    plt.legend()
    plt.tight_layout()
    plt.show()

## Takeaways

### 2. Is the price of the house influenced by the county/ location?
#### It appears that houses are less expensive in LA and than in Ventura and Orange county respectively
#### However this does not seem like a strong relationship with the price, therefore this is a feature will be  explored after presenting my current findings



### 3. Does the age of the house matter when it comes to the price? 


In [ ]:
#this codetakes in a sample of the data and creates a scatter plot with hue that showes where houses
#in the sample are located and draws a regression line
sns.lmplot(x="yearbuilt", y="total_taxes", hue ="county", data=train.sample(1000))


## Takeaways
### 3. Does the age of the house matter when it comes to the price? 
#### The houses in Ventura and in Orange county appear to be more expensive the newer they are, unlike in Los angeles

### 4. Does the size of the house in sqfeet influence the price?


In [ ]:
#here is a comparisson between squarefeet and total_taxes with the colors indicating what county the houses are in(picked a sample of 1000) 

sns.lmplot(x="squarefeet", y="total_taxes", hue ="county", data=train.sample(1000))


## Takeaways
### 4. Does the size of the house matter when it comes to the price? 
#### it appears that the size of the house has a high positive relationship with the price. this is also evident with the positive relationship between the number of bedrooms and total_taxes


## Summary 

#### 1. Does the number of bedrooms and bathrooms in a house increase the cost of the home
- It appears that the number of bedrooms have a positive relationship with the cost of the price of house
- Additionally according to the charts the higher the number of bathrooms the higher the price

### 2. Is the price of the house influenced by the county/ location?
- It appears that houses are less expensive in LA and than in Ventura and Orange county respectively
- However this does not seem like a strong relationship with the price, therefore this feature will not be  explored in this report. I will revisit after

### 3. Does the age of the house matter when it comes to the price? 
- The houses in Ventura and in Orange county appear to be more expensive the newer they are, unlike in Los angeles

### 4. Does the size of the house matter when it comes to the price? 
- It appears that the size of the house has a high positive relationship with the price. This is also evident with the positive relationship between the number of bedrooms and total_taxes

#### In summary I will explore statistical relationships in bathrooms, bedrooms, the year built and square feet(size)

## Statistical tests

In [ ]:
a = 0.05

### Hypothesis Testing
- In this section i will be checking the correlation between the variables that visually showed a relationship with the total_tax column
- Afterwards, I will proceed to pick 3 of the best variables to model with

#### $ H0 $ : the number of bathrooms have no relationship to the property prices

#### $ H1 $ : the higher the number of bathrooms, the higher the price of the house 

In [ ]:
# here I will use Pearsons r to check the correlation between the variables 
x = train.bathrooms
y = train.total_taxes

corr, p = stats.pearsonr(x, y)
print(f'corr = {corr:.5f}')
print(f'p = {p:.5f}')

if p<a:
    print(f"I reject the null hypothesis")
else:
    print(f"I reject the null hypothesis")


#### $ H0 $ : the number of bedrooms have no relationship to the property prices
#### $ H1 $ : the higher the number of bedrooms, the higher the price of the house 

In [ ]:
# this code takes in the number of bedrooms and the total taxes 
#returns correlation and p values to determine the statistical significance
x1 = train.bedrooms
y1 = train.total_taxes

corr1, p1 = stats.pearsonr(x1, y1)
print(f'corr = {corr1:.5f}')
print(f'p = {p1:.5f}')

if p<a:
    print(f"I reject the null hypothesis")
else:
    print(f"I reject the null hypothesis")


#### $H0$: the age of the house has no relationship to the property prices

#### $H1$:the newer the property the more expensive it is 

In [ ]:
# this code takes in the year the property was built and the total taxes 
#returns r and p values to determine the statistical significance
x3= train["yearbuilt"]
y3 = train["total_taxes"]
r, p2 = stats.pearsonr(x3, y3)
print(f'r = {r:.5f}')
print(f'p = {p2:.5f}')
# however it is a weak correlation 
if p<a:
    print(f"I reject the null hypothesis")
else:
    print(f"I reject the null hypothesis")

#### $H0$: the size of the house in squarefeet does not have a relationship with the price

#### $H1$: the size of the house in squarefeet has a relatively strong correlation with the property prices

In [ ]:
# this code takes in the squarefeet and the total taxes 
#returns r and p values to determine the statistical significance

x4= train["squarefeet"]
y4 = train["total_taxes"]
r1, p3 = stats.pearsonr(x4, y4)
print(f'r = {r1:.5f}')
print(f'p = {p3:.5f}')

if p<a:
    print(f"I reject the null hypothesis")
else:
    print(f"I reject the null hypothesis")

In [ ]:
#this code illustrates the correlation matrix where correlations are observed among different variables 
cor = train.corr()
cor

In [ ]:
#here is a plot of the correlation coefficients 
plt.figure(figsize=(10,8))
sns.heatmap(cor,annot = True,cmap = 'mako',center = 0,vmin = -1, vmax = 1)

### Summary
- All of the variables had a positive correlation
- However I will proceed to model with the three strongest correlated variables 
- These are:
    - bedrooms
    - bathrooms 
    - squarefeet
    

# Scaling

In [ ]:
# This function takes in a data set that is split , makes a copy and uses the min max scaler 
# to scale all three data sets. additionally it adds the columns names on the scaled data and 
# returns trainedscaled data, validate scaled data and test scale

columns_to_scale = ["bedrooms","bathrooms","squarefeet"]
train_scaled, validate_scaled,test_scaled =prepare.scaling_zillow(train, validate, test, columns_to_scale)
train_scaled.head()

## BASELINE


In [ ]:
# We need y_train and y_validate to be dataframes to append the new columns with predicted values. 
# y_train = pd.DataFrame(y_train)
# y_validate = pd.DataFrame(y_validate)

X_train, y_train = train_scaled, train_scaled["total_taxes"]
X_validate, y_validate = validate_scaled, validate_scaled.total_taxes
X_test, y_test = test_scaled, test_scaled.total_taxes
y_train = pd.DataFrame(train.total_taxes)
y_validate = pd.DataFrame(validate.total_taxes)
y_test = pd.DataFrame(test.total_taxes)


# 1. Predict housing pices 
y_train_mean = y_train['total_taxes'].mean()
y_train['y_train_mean'] = y_train_mean
y_validate['y_train_mean'] = y_train_mean

# 2. compute prices prediction
y_train_median = y_train['total_taxes'].median()
y_train['y_train_median'] = y_train_median
y_validate['y_train_median'] = y_train_median

# 3. RMSE for mean
rmse_train = mean_squared_error(y_train.total_taxes, y_train.y_train_mean)**(1/2)
rmse_validate = mean_squared_error(y_validate.total_taxes, y_validate.y_train_mean)**(1/2)

print("RMSE using Mean\nTrain/In-Sample: ", round(rmse_train, 2), 
            "\nValidate/Out-of-Sample: ", round(rmse_validate, 2))

# 4. RMSE for median
rmse_train = mean_squared_error(y_train.total_taxes, y_train.y_train_median)**(1/2)
rmse_validate = mean_squared_error(y_validate.total_taxes, y_validate.y_train_median)**(1/2)

print("RMSE using Median\nTrain/In-Sample: ", round(rmse_train, 2), 
                  "\nValidate/Out-of-Sample: ", round(rmse_validate, 2))

## MODELING (TweedieRegressor)

In [ ]:
# create the model object
glm = TweedieRegressor(power=1, alpha=0)
X_train_tr = X_train[["bedrooms","bathrooms","squarefeet"]]
X_validate_tr = X_validate[["bedrooms","bathrooms","squarefeet"]]

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
glm.fit(X_train_tr, y_train.total_taxes)

# predict train
y_train['total_taxes_pred'] = glm.predict(X_train_tr)

# evaluate: rmse
rmse_train_tr = mean_squared_error(y_train.total_taxes, y_train.total_taxes_pred)**(1/2)

# predict validate
y_validate['total_taxes_pred'] = glm.predict(X_validate_tr)

# evaluate: rmse
rmse_validate_tr = mean_squared_error(y_validate.total_taxes, y_validate.total_taxes_pred)**(1/2)

print("RMSE for GLM using Tweedie, power=1 & alpha=0\nTraining/In-Sample: ", rmse_train_tr, 
      "\nValidation/Out-of-Sample: ", rmse_validate_tr)


## MODELING (features: bedrooms, bathrooms, squarefeet)
#### Best model is a linear regression with 3 features

In [ ]:
# create the model object
lm = LinearRegression(normalize=True)
X_train = X_train[["bedrooms","bathrooms","squarefeet"]]
X_validate = X_validate[["bedrooms","bathrooms","squarefeet"]]
# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
lm.fit(X_train, y_train.total_taxes)

# predict train
y_train['total_taxes_pred'] = lm.predict(X_train)

# evaluate: rmse
rmse_train_lr = mean_squared_error(y_train.total_taxes, y_train.total_taxes_pred)**(1/2)

# predict validate
y_validate['total_taxes_pred'] = lm.predict(X_validate)

# evaluate: rmse
rmse_validate_lr = mean_squared_error(y_validate.total_taxes, y_validate.total_taxes_pred)**(1/2)

print("RMSE for OLS using LinearRegression\nTraining/In-Sample: ", rmse_train_lr, 
      "\nValidation/Out-of-Sample: ", rmse_validate_lr)

### Summary
- the linear regression shows the most promise with these three features.
- however there are concerns that these features are very similar or the bedrooms and the bathrooms are contained int the sqfeet
- so far this is the best model

### Baseline

RMSE using Mean

Train/In-Sample:  264487.8

Validate/Out-of-Sample:  264650.06

RMSE using Median

Train/In-Sample:  268181.81

Validate/Out-of-Sample:  268086.55

### Linear regression

RMSE for OLS using LinearRegression

Training/In-Sample:  226874.71769377927 

Validation/Out-of-Sample:  224745.86491873223





In [ ]:
RMSE_baseline_train = rmse_train 
RMSE_baseline_validate = rmse_validate
RMSE_train_lr = rmse_train_lr
RMSE_validate_lr = rmse_validate_lr
print (f"Difference with Baseline Train = {RMSE_baseline_train-RMSE_train_lr}")
print(f"Difference with Baseline Validate = {RMSE_baseline_validate-RMSE_validate_lr}")

### Conclusion
In conclusion, the model was able to beat the baseline by atleast $40000.

However, some of the features involved seem to have similar effect on the model.
The correlation coefficient of bathrooms and squarefeet was at 81. 

In the future, we can include some feature engineering where we would combine sqfeet, bedrooms and bathrooms to make one feature.

Additionally, with more time we can run other regression models to make the process more accurate 